<a href="https://colab.research.google.com/github/profteachkids/CHE3022_Fall2022/blob/main/NetworkFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
from scipy.optimize import root

In [4]:
mu = 1e-3  #Pa s
rho = 1e3   # kg/m3
g=9.81      #m/s2

In [5]:
def churchill(Re, eD):
    t1 = (-2.457*np.log((7./Re)**0.9 + 0.27*eD))**16
    t2 = (37530/Re)**16
    return 2* ((8/Re)**12 + 1/(t1+t2)**1.5)**(1/12.)

In [8]:
def headDrop(Q, L, D, eD):
   v=4*Q/(np.pi*(D**2))
   Re = rho * v * D / mu
   f = churchill(Re, eD)
   return 2*f*L*(v**2)/(g*D)

In [7]:
dHead(0.1, 10, 0.1, 5e-3)

25.182198539291203